# setup

## write dsub file

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  #local AOU_NETWORK=network
  #local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-batch \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'ubuntu:latest' \
      --network "global/networks/network" \
      --subnetwork "regions/us-central1/subnetworks/subnetwork" \
      --service-account "$(gcloud config get-value account)" \
      --use-private-address \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

# subset plink files to variants in scores and individuals in sample

## copy plink files to workspace bucket

In [ ]:
!gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/* ${WORKSPACE_BUCKET}/HF/input/acaf/

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls -lh gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/*

## run command

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'plink_subset-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env CHROM': [x for x in range(1,23)],
    '--output-recursive OUT_DIR': [f"{bucket}/HF/input/acaf_pgen_subset/" for _ in range(22)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/plink_subset" \
  --disk-size 180 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --pfile $BUCKET/HF/input/acaf/acaf_threshold.chr$CHROM \
                      --extract range $BUCKET/HF/input/HF.PGS005097.var_list.txt \
                      --set-all-var-ids @:#:\$r:$a \
                      --new-id-max-allele-len 1000 \
                      --make-pgen \
                      --out $OUT_DIR/acaf_threshold.hf_pgs_weights_vars.chr$CHROM'

print("\n".join(job_output))
plink_job_id = job_output[1].split(" ")[-1]
%env PLINK_JOB_ID={plink_job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "plink-subs--kathleen-cardone--250702-175543-72" \
    --users "kathleen-cardone" \
    --status '*'

## check outputs

In [ ]:
!gsutil ls -lh ${WORKSPACE_BUCKET}/HF/input/acaf_pgen_subset/

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/HF/input/acaf_pgen_subset/acaf_threshold.hf_pgs_weights_vars.chr1.psam .
!wc -l acaf_threshold.hf_pgs_weights_vars.chr1.psam

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/HF/input/acaf_pgen_subset/acaf_threshold.hf_pgs_weights_vars.chr1.pvar .
!wc -l acaf_threshold.hf_pgs_weights_vars.chr1.pvar

## remove plink files from workspace bucket

In [ ]:
!gsutil rm ${WORKSPACE_BUCKET}/HF/input/acaf/*

# create sample sheet for PGSC-CALC

## create file path list

In [ ]:
filepath_list = []
for chr in list(range(1,23)):
    filepath = '/mnt/data/mount/gs/fc-secure-b117990b-e93e-442c-a994-81f2ca68d4fb/HF/input/acaf_pgen_subset/acaf_threshold.hf_pgs_weights_vars.chr' + str(chr)
    filepath_list.append(filepath)
print(len(filepath_list))
filepath_list

## create dataframe

In [ ]:
import pandas as pd
samplesheet = pd.DataFrame(data={'sampleset':'AOU',
                                'path_prefix':filepath_list,
                                'chrom':list(range(1,23)),
                                'format':'pfile'})
samplesheet

## export dataframe

In [ ]:
samplesheet.to_csv('AOU.HF.PGSC_CALC.samplesheet.csv',index=None)

## copy to workspace bucket

In [ ]:
!gsutil cp AOU.HF.PGSC_CALC.samplesheet.csv ${WORKSPACE_BUCKET}/HF/input/

# run PGSC-calc

## run command

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'pgsc_calc'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--output-recursive OUT_DIR': [f"{bucket}/HF/output/"]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/pgsc_calc_general:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/hf/" \
  --disk-size 512 \
  --boot-disk-size 20 \
  --min-ram 208 \
  --min-cores 32 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'nextflow run pgscatalog/pgsc_calc -profile conda \
                  --input $BUCKET/CKD/input/AOU.CKD.PGSC_CALC.samplesheet.csv \
                  --scorefile "$BUCKET/HF/input/pgs_weights/*" \
                  --target_build GRCh38 \
                  --outdir $OUT_DIR \
                  --max_cpus 32 \
                  --max_memory 208.GB \
                  --min_overlap 0.0 \
                  --max_time 240.h \
                  --run_ancestry $BUCKET/CKD/input/pgsc_HGDP+1kGP_v1.tar.zst \
                  --keep_multiallelic True \
                  --hwe_ref 0 \
                  --pca_maf_target 0.05'

job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "pgsc-calc--kathleen-cardone--250703-010752-01" \
    --users "kathleen-cardone" \
    --status '*'

## check logs

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/dsub_logs/hf/pgsc-calc--kathleen-cardone--250703-010752-01* .

In [ ]:
!cat pgsc-calc--kathleen-cardone--250703-010752-01.1-stderr.log

In [ ]:
!cat pgsc-calc--kathleen-cardone--250703-010752-01.1-stdout.log

# IRM feature selection

## copy script and inputs

In [ ]:
!gsutil cp hf_training_script.py ${WORKSPACE_BUCKET}/HF/scripts/

In [ ]:
!gsutil cp in_progress_pheno_no_missing.csv ${WORKSPACE_BUCKET}/HF/input/

In [ ]:
!gsutil cp HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv ${WORKSPACE_BUCKET}/HF/input/

## submit first 500 jobs (to avoid exceeding quota)

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'regression'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ITER': list(range(1, 501)),
    '--output-recursive OUT_DIR': [f"{bucket}/HF/output/regressions/train/" for _ in range(500)]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

print('submitting job')

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/python:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/hf/regressions/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'python "${BUCKET}"/HF/scripts/hf_training_script.py \
                --input "${BUCKET}"/HF/input/HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv \
                --iter "${ITER}" \
                --output_dir "${OUT_DIR}"/'

print("\n".join(job_output))
job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "regression--kathleen-cardone--250922-223405-86" \
    --users "kathleen-cardone" \
    --status '*'

## check output

In [ ]:
!gsutil ls ${WORKSPACE_BUCKET}/HF/output/regressions/train/ | wc -l

## submit next 500 jobs

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'regression'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ITER': list(range(501, 1001)),
    '--output-recursive OUT_DIR': [f"{bucket}/HF/output/regressions/train/" for _ in range(500)]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

print('submitting job')

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/python:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/hf/regressions/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'python "${BUCKET}"/HF/scripts/hf_training_script.py \
                --input "${BUCKET}"/HF/input/HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv \
                --iter "${ITER}" \
                --output_dir "${OUT_DIR}"/'

print("\n".join(job_output))
job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "regression--kathleen-cardone--250922-224900-98" \
    --users "kathleen-cardone" \
    --status '*'

## check output

In [ ]:
!gsutil ls ${WORKSPACE_BUCKET}/HF/output/regressions/train/ | wc -l

# evaluate IRM performance

## copy inputs and script

In [ ]:
!gsutil cp hf_eval_script.py ${WORKSPACE_BUCKET}/HF/scripts/

In [ ]:
!gsutil -m cp significant_vars_95.csv important_vars_95.csv LR_beta_all_iter.csv HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv ${WORKSPACE_BUCKET}/HF/input/

## submit first 500 jobs

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'eval'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ITER': list(range(1, 501)),
    '--output-recursive OUT_DIR': [f"{bucket}/HF/output/regressions/eval/" for _ in range(500)]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

print('submitting job')

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/python:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/hf/regressions/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'python "${BUCKET}"/HF/scripts/hf_eval_script.py \
                --input "${BUCKET}"/HF/input/HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv \
                --sig "${BUCKET}"/HF/input/significant_vars_95.csv \
                --important "${BUCKET}"/HF/input/important_vars_95.csv \
                --beta "${BUCKET}"/HF/input/LR_beta_all_iter.csv \
                --iter "${ITER}" \
                --output_dir "${OUT_DIR}"/'

print("\n".join(job_output))
job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "eval--kathleen-cardone--250923-180411-31" \
    --users "kathleen-cardone" \
    --status '*'

## check output

In [ ]:
!gsutil ls ${WORKSPACE_BUCKET}/HF/output/regressions/eval/ | wc -l

## submit second 500 jobs

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'eval'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ITER': list(range(501, 1001)),
    '--output-recursive OUT_DIR': [f"{bucket}/HF/output/regressions/eval/" for _ in range(500)]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

print('submitting job')

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/python:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/hf/regressions/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'python "${BUCKET}"/HF/scripts/hf_eval_script.py \
                --input "${BUCKET}"/HF/input/HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv \
                --sig "${BUCKET}"/HF/input/significant_vars_95.csv \
                --important "${BUCKET}"/HF/input/important_vars_95.csv \
                --beta "${BUCKET}"/HF/input/LR_beta_all_iter.csv \
                --iter "${ITER}" \
                --output_dir "${OUT_DIR}"/'

print("\n".join(job_output))
job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "eval--kathleen-cardone--250923-181826-58" \
    --users "kathleen-cardone" \
    --status '*'

## check output

In [ ]:
!gsutil ls ${WORKSPACE_BUCKET}/HF/output/regressions/eval/ | wc -l